The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [4]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [5]:
%sql postgresql://chrble@/lustre


u'Connected: chrble@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [6]:
%sql select * from lustre limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
usr356,i5105,1360399005,1368700859,4,2248,/scratch/bioc/usr356/param/ZINC04992888.rtf
usr356,i5105,1360387170,1368792474,8,6640,/scratch/bioc/usr356/param/ZINC71832434.prm
usr356,i5105,1360416215,1368785748,8,4480,/scratch/bioc/usr356/param/ZINC71850771.prm
usr356,i5105,1360404430,1368792486,4,2556,/scratch/bioc/usr356/param/ZINC34729086.rtf
usr356,i5105,1360420861,1368774402,4,2065,/scratch/bioc/usr356/param/ZINC45891658.rtf


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [ ]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [7]:
%%time

import time
time.sleep(5)

CPU times: user 845 µs, sys: 0 ns, total: 845 µs
Wall time: 5.01 s


----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.  Look at the documentation for the [ALTER TABLE](https://www.tutorialspoint.com/sql/sql-alter-command.htm) statement.

We shall break this down into steps: (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [21]:
# (1) create a new table and populate it
%sql CREATE TABLE new_table(LIKE lustre_sample);

(psycopg2.ProgrammingError) relation "new_table" already exists
 [SQL: 'CREATE TABLE new_table(LIKE lustre_sample);']


In [22]:
# (2) populate it with a copy of the data
%sql INSERT INTO new_table SELECT * FROM lustre_sample;

1519053 rows affected.


[]

In [23]:
# (3) alter table definition: add new columns for "access time" using the TIMESTAMP type
%sql ALTER TABLE new_table ADD COLUMN access_time timestamp;

(psycopg2.ProgrammingError) column "access_time" of relation "new_table" already exists
 [SQL: 'ALTER TABLE new_table ADD COLUMN access_time timestamp;']


In [25]:
# (4) populate additional column
%sql UPDATE new_table SET access_time = to_timestamp(atime);


1519053 rows affected.


[]

Show some data from the table we created::

In [26]:
%sql select * from new_table limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path,access_time
usr25,i5105,1382131058,1363059494,880820,901952052,/scratch/bioc/usr25/fegs/b3s/fegscoor/fegs78/coor56.xtc,2013-10-18 21:17:38
usr25,i5105,1374746779,1374746779,4,111,/scratch/bioc/usr25/mddock/y3ma_059/t1.sh,2013-07-25 10:06:19
usr25,uzh,1366377914,1178206881,4,2127,/scratch/bioc/usr25/soft/SuiteSparse/CAMD/Doc/docdiff,2013-04-19 13:25:14
usr25,uzh,1367648149,1362274204,280,283680,/scratch/bioc/usr25/soft/igraph-0.6.5/ltmain.sh,2013-05-04 06:15:49
usr25,uzh,1367648896,1366376801,4,2217,/scratch/bioc/usr25/soft/Cython-0.19/README.txt,2013-05-04 06:28:16


## 2. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

In [27]:
%sql SELECT count(DISTINCT path) FROM lustre_sample WHERE path LIKE '/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/%';

1 rows affected.


count
310230


## 3. Can you find the directory that holds the largest number of files? 

In [28]:
%sql SELECT count(dir) AS count,dir FROM (SELECT substring(path FROM '(.+)\/.+\.pdb') AS dir FROM lustre_sample) AS dirs GROUP BY dir ORDER BY count DESC LIMIT 5;

5 rows affected.


count,dir
310230,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose
111032,/scratch/bioc/usr356/dock_parent_3P1C_w3
96952,/scratch/bioc/usr356/pdbqt
96416,/scratch/bioc/usr356/vina_3P1C_w3
96291,/scratch/bioc/usr356/vina_3SVH_w3


## 4. Can you find the directory tree that holds the largest number of files?

Yes or no? *(and why?)*

Yes, theoretically with nested sets. In practice it would be too complicated.